# title.principals.tsv.gz 

https://datasets.imdbws.com/title.principals.tsv.gz

– Contains the principal cast/crew for titles

* tconst (string) - alphanumeric unique identifier of the title
* ordering (integer) – a number to uniquely identify rows for a given titleId
* nconst (string) - alphanumeric unique identifier of the name/person
* category (string) - the category of job that person was in
* job (string) - the specific job title if applicable, else '\N'
* characters (string) - the name of the character played if applicable, else '\N'

```
CREATE TABLE `title.principals` (
  `tconst` varchar(12) NOT NULL,
  `ordering` int(11) NOT NULL,
  `nconst` varchar(12),
  `category` text,
  `job` text,
  `characters` text
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
```
#### import the title.principals.tsv file
```
load data infile '/export/title.principals.tsv' 
    into table `title.principals` 
    fields terminated by '\t' 
    lines terminated by '\n' 
    ignore 1 rows;
```
* Query OK, 46817295 rows affected (10 min 34.591 sec)
* Records: 46817295  Deleted: 0  Skipped: 0  Warnings: 0


In [11]:
import mysql.connector
# widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# files
import os
import re
import shutil
import urllib.parse

 

import IMDBmedia as IM

IM.imdb['host'] = "localhost"
IM.imdb['user'] = "root"
IM.imdb['pass'] = "root"
IM.imdb['dbname'] = "IMDBmedia"

In [32]:
sql="""
SELECT * FROM `title.principals` 
WHERE category LIKE 'act%'
LIMIT 15;
"""
val=()
List = [
    ('Vin Diesel', 'nm0004874' ),
    ('Jet Li', 'nm0001472'),
    ('Brad Pitt', 'nm0000093')
]
SDB=IM.search_db(sql, val)
for x in SDB:
    List.append( ( x[2], x[2] ) )
    print(x)

('tt0000005', 1, 'nm0443482', 'actor', None, '["Blacksmith"]')
('tt0000005', 2, 'nm0653042', 'actor', None, '["Assistant"]')
('tt0000007', 1, 'nm0179163', 'actor', None, None)
('tt0000007', 2, 'nm0183947', 'actor', None, None)
('tt0000008', 1, 'nm0653028', 'actor', None, '["Sneezing Man"]')
('tt0000009', 1, 'nm0063086', 'actress', None, '["Miss Geraldine Holbrook (Miss Jerry)"]')
('tt0000009', 2, 'nm0183823', 'actor', None, '["Mr. Hamilton"]')
('tt0000009', 3, 'nm1309758', 'actor', None, '["Chauncey Depew - the Director of the New York Central Railroad"]')
('tt0000011', 1, 'nm3692297', 'actor', None, '["Acrobats"]')
('tt0000014', 1, 'nm0166380', 'actor', None, '["The Gardener"]')
('tt0000014', 2, 'nm0244989', 'actor', None, '["The Boy"]')
('tt0000017', 1, 'nm3691272', 'actor', None, '["The boy"]')
('tt0000017', 2, 'nm3692829', 'actress', None, '["The girl"]')
('tt0000018', 1, 'nm3692071', 'actor', None, '["The Boxer"]')
('tt0000026', 1, 'nm2350007', 'actor', None, '["Valet"]')


# Example of movies
Using the `nconst` for Jackie Chan, this will list all his movies as an Actor

In [31]:
def check_my_list(Act):
    print(Act)
    sql="""
SELECT 
    P.tconst as ID, ML.title, ML.year, ML.file, ML.url
FROM `title.principals` P, movies_list ML
WHERE ML.titleId = P.tconst 
    AND P.nconst = '""" + Act + "'"
    val=(Act)
    SDB=IM.search_db(sql, val)
    r = len(SDB)
    if (r > 0):
        for x in SDB:
            print(x)
    else:
        print("No records found")
interact_manual(check_my_list, Act=List);

interactive(children=(Dropdown(description='Act', options=(('Vin Diesel', 'nm0004874'), ('Jet Li', 'nm0001472'…

# Exampes of quries that didn't work out
These quries produced incorrect results and took a lot of time to execute.

format of the data bellow

* SQL query
* row (time)

```
SELECT DISTINCT     N.nconst, N.primaryName, P.category, count(P.tconst)  FROM `title.principals` P, `name.basics` N WHERE P.nconst = N.nconst AND P.tconst IN (Select titleID from movies_list) Group by P.tconst Having count(P.tconst) > 3 Order by count(P.tconst) DESC;
```
**417 rows in set (1 hour 33 min 8.305 sec)**

```
SELECT DISTINCT     N.nconst, N.primaryName, count(P.tconst)  FROM `title.principals` P, `name.basics` N  WHERE P.nconst = N.nconst AND P.tconst IN (Select titleID from movies_list) AND (P.category = 'act%' OR  P.category = 'self') Group by P.tconst Having count(P.tconst) > 3 Order by count(P.tconst) DESC;
```
**9 rows in set (20 min 25.451 sec)**
```
SELECT  N.nconst, N.primaryName, count(P.tconst)  FROM `title.principals` P, `name.basics` N  WHERE P.nconst = N.nconst AND P.tconst IN (Select titleID from movies_list) AND (P.category = 'act%' OR  P.category = 'self') Group by P.tconst Having count(P.tconst) > 3 Order by count(P.tconst) DESC;
```
**10 rows in set (20 min 31.821 sec)**
```
 SELECT  N.nconst, N.primaryName, count(M.titleID)  FROM `title.principals` P, `name.basics` N, movies_list M  WHERE P.nconst = N.nconst AND P.tconst = M.titleID AND (P.category = 'act%' OR  P.category = 'self') Group by N.primaryName Having count(M.titleID) > 3 Order by count(M.titleID) DESC;
```
**Empty set (21 min 17.782 sec)**
```
SELECT  N.nconst, N.primaryName, M.file  FROM `title.principals` P, `name.basics` N, movies_list M  WHERE P.nconst = N.nconst AND P.tconst = M.titleID AND (P.category = 'act%' OR  P.category = 'self') ;
```
**54 rows in set (21 min 41.241 sec)**


In [35]:
sql="""
SELECT DISTINCT year FROM movies_list 
Order by year DESC
LIMIT 15;
"""
val=()
Years = []
SDB=IM.search_db(sql, val)
for x in SDB:
    Years.append( x[0] )
    print(x)

('2021',)
('2020',)
('2019',)
('2018',)
('2017',)
('2016',)
('2015',)
('2014',)
('2013',)
('2012',)
('2011',)
('2010',)
('2009',)
('2008',)
('2007',)


In [41]:
def movies_by_year(Year):
    sql="SELECT * FROM movies_list WHERE year LIKE " + Year
    val=(Year)
    SDB=IM.search_db(sql, val)
    r = len(SDB)
    if (r > 0):
        for x in SDB:
            Movies.append( ( x[1] + " (" + x[2] + ")", x[0] ) )
            print(x[1] + " (" + x[2] + ")")
    else:
        print("No Movies found")
Movies = []
interact_manual(movies_by_year, Year=Years);

interactive(children=(Dropdown(description='Year', options=('2021', '2020', '2019', '2018', '2017', '2016', '2…

In [47]:
def actors_from_movies(Movie):
    print(Movie)
    sql="SELECT  N.nconst, N.primaryName, P.category FROM `name.basics` N INNER JOIN `title.principals` P ON P.nconst = N.nconst WHERE P.tconst LIKE '" + Movie + "';" 
    val=(Movie)
    SDB=IM.search_db(sql, val)
    r = len(SDB)
    if (r > 0):
        for x in SDB:
            Actors.append( ( x[1] , x[0] ) )
            print(x[1],"\nhttps://www.imdb.com/name/" + x[0] + "/")
    else:
        print("No Acters found")
Actors = []
interact_manual(actors_from_movies, Movie=Movies);

interactive(children=(Dropdown(description='Movie', options=(('Blended (2014)', 'tt1086772'), ('RoboCop (2014)…

In [ ]:
sql = """SELECT DISTINCT N.nconst, N.primaryName, count(P.tconst) 
FROM `title.principals` P, `name.basics` N 
WHERE P.nconst = N.nconst 
    AND P.category Like 'Act%' 
    AND P.tconst IN (Select titleID from movies_list) 
Group by N.nconst 
Having count(P.tconst) > 3 
Order by count(P.tconst) DESC
"""